In [1]:
from sklearn.datasets import (
    load_iris, load_wine,
    load_digits, load_breast_cancer
)
import pandas as pd
import os, sys
# asumiendo que iris.ipynb está en notebooks/, y somJ/ en la carpeta superior:
sys.path.insert(0, os.path.abspath(".."))

from somJ.k_fold import cross_val_som_metrics
import somJ.config as config

datasets = {
    'Iris':           load_iris(),
    'Wine':           load_wine(),
    'Digits':         load_digits(),
    'Breast Cancer':  load_breast_cancer()
}

for name, ds in datasets.items():
    X, y = ds.data, ds.target
    metrics = cross_val_som_metrics(
        X, y,
        k=5,
        method="pca",
        update="batchmap",
        config=config,
        random_state=42,
        scale=True
    )
    print(
        f"{name:15s} | "
        f"Acc: {metrics['accuracy']:.2f}  | "
        f"Prec: {metrics['precision']:.2f}  | "
        f"Rec: {metrics['recall']:.2f}  | "
        f"F1: {metrics['f1']:.2f}"
    )


Iris            | Acc: 0.99  | Prec: 0.99  | Rec: 0.99  | F1: 0.99
Wine            | Acc: 0.98  | Prec: 0.98  | Rec: 0.98  | F1: 0.98
Digits          | Acc: 0.97  | Prec: 0.97  | Rec: 0.97  | F1: 0.97
Breast Cancer   | Acc: 0.98  | Prec: 0.97  | Rec: 0.97  | F1: 0.97
